In [1]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 106.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 100.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 90.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.5/206.5 kB

In [2]:
!git clone https://github.com/AI4Bharat/IndicBERT

Cloning into 'IndicBERT'...
remote: Enumerating objects: 500, done.
remote: Counting objects: 100% (170/170), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 500 (delta 93), reused 125 (delta 58), pack-reused 330
Receiving objects: 100% (500/500), 148.26 KiB | 7.80 MiB/s, done.
Resolving deltas: 100% (252/252), done.


In [16]:
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("ai4bharat/indic-bert", num_labels=5)
tokenizer = AutoTokenizer.from_pretrained('ai4bharat/indic-bert' )

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertForSequenceClassification: ['sop_classifier.classifier.bias', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.decoder.weight', 'predictions.bias', 'predictions.decoder.bias', 'sop_classifier.classifier.weight', 'predictions.LayerNorm.weight']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indi

In [17]:
import pandas as pd
train = pd.read_csv('train.csv')

In [18]:
train.head()

,id,polarity,text
0,1,conflict,"रैमो डिसूजा की फिल्म ‘एबीसीडी 2’ में डांस है, ..."
1,2,neutral,दृश्ये और प्रसंग के हिसाब से इन भाव और भावनाओं...
2,3,neutral,फिल्म सच्ची कहानी पर आधारित है तो कुछ सच्चाई ...
3,4,positive,मुश्किल स्टेप के अच्छेे डांस सिक्वेंस हैं।
4,6,neutral,पास बैठे युवा दर्शक की टिप्पणी थी कि इन दिनों ...


In [19]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

polarity = train['polarity']

polarity = le.fit_transform(list(polarity))

In [20]:
len(set(polarity))

5

In [21]:
text = list(train['text'])

In [30]:
set(train['polarity'])

{'conflict', 'cpn', 'negative', 'neutral', 'positive'}

In [32]:
train.columns

Index(['id', 'polarity', 'text'], dtype='object')

In [34]:
train.drop(['id'], axis=1, inplace=True)

In [39]:
# train = train.rename(columns={"polarity": "label"}, errors="raise")
train = train.rename(columns={"label": "labels"}, errors="raise")

In [44]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

model_args = ClassificationArgs()
model_args.num_train_epochs = 3
# model_args.learning_rate = 1e-5
model_args.train_batch_size = 16
model_args.labels_list = ["conflict", "cpn", "negative", 'neutral', 'positive']
model_args.max_seq_length = 512
model_args.overwrite_output_dir = True
# Add more configuration options as needed

model = ClassificationModel(
    "albert", "ai4bharat/indic-bert", num_labels = 5, args=model_args, use_cuda=True
)

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertForSequenceClassification: ['sop_classifier.classifier.bias', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.decoder.weight', 'predictions.bias', 'predictions.decoder.bias', 'sop_classifier.classifier.weight', 'predictions.LayerNorm.weight']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indi

In [45]:
train.columns

Index(['labels', 'text'], dtype='object')

In [46]:
model.train_model(train)

  0%|          | 0/2052 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/129 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/129 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/129 [00:00<?, ?it/s]

(387, 1.1074228015672944)

In [48]:
test = pd.read_csv('/content/shuffled_test.csv')

In [49]:
predictions, raw_outputs = model.predict(test["text"].values.tolist())

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [50]:
predictions

['negative',
 'neutral',
 'positive',
 'positive',
 'negative',
 'positive',
 'positive',
 'negative',
 'neutral',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'positive',
 'neutral',
 'neutral',
 'negative',
 'positive',
 'positive',
 'negative',
 'negative',
 'positive',
 'positive',
 'neutral',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'positive',
 'neutral',
 'negative',
 'positive',
 'positive',
 'neutral',
 'negative',
 'positive',
 'positive',
 'negative',
 'negative',
 'neutral',
 'negative',
 'negative',
 'neutral',
 'positive',
 'negative',
 'positive',
 'negative',
 'neutral',
 'neutral',
 'negative',
 'positive',
 'negative',
 'negative',
 'neutral',
 'negative',
 'negative',
 'positive',
 'neutral',
 'negative',
 'positive',
 'negative',
 'positive',
 'positive',
 'negative',
 'negative',
 'neutral',
 '

In [51]:
from sklearn.metrics import classification_report

In [53]:
labels = test['polarity']

In [54]:
labels

0     positive
1      neutral
2     positive
3     positive
4      neutral
        ...   
95     neutral
96    conflict
97    positive
98    conflict
99    negative
Name: polarity, Length: 100, dtype: object

In [55]:
report = classification_report(labels.values.tolist(), predictions)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [56]:
print(report)

              precision    recall  f1-score   support

    conflict       0.00      0.00      0.00        25
    negative       0.46      0.88      0.60        25
     neutral       0.54      0.52      0.53        25
    positive       0.61      0.68      0.64        25

    accuracy                           0.52       100
   macro avg       0.40      0.52      0.44       100
weighted avg       0.40      0.52      0.44       100

